In [19]:
from importlib import reload  
import models
import gtzan_dataset

reload(gtzan_dataset)
reload(models)

<module 'models' from '/workspaces/gtzan-classification/scripts/models.py'>

In [20]:
import pytorch_lightning as pl
from models import SpectogramClassifier
from gtzan_dataset import GTZANDataset
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

In [21]:
batch_size = 1

In [35]:
dataset = GTZANDataset(csv_file='../processed_data/train_data.csv')

In [36]:
from sklearn.model_selection import KFold

In [37]:
k_fold = KFold(n_splits=4)

In [38]:
input_shape = dataset[0][0].shape

In [39]:
val_length = int(0.3 * len(dataset))
train_length = len(dataset) - val_length

train_set, val_set = random_split(dataset, [train_length, val_length])

In [40]:
input_shape

torch.Size([1, 128, 1293])

In [41]:
classifier_model = SpectogramClassifier(input_shape=input_shape, num_classes=dataset.labels_count, learning_rate=0.001)

In [42]:
# Test dimensions
classifier_model(dataset[0][0].unsqueeze(0))

tensor([[-2.2840, -2.2419, -2.2269, -2.3473, -2.3002, -2.2712, -2.3527, -2.3204,
         -2.3382, -2.3527]], grad_fn=<LogSoftmaxBackward>)

In [43]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [44]:
val_loader = DataLoader(val_set, batch_size=batch_size)

In [32]:
# saves a file like: my/path/sample-mnist-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor="validation_loss",
    dirpath="best_model/",
    save_top_k=1,
    mode="min",
)

In [45]:
logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = pl.Trainer(logger=logger, max_epochs=15, callbacks=[checkpoint_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [46]:
trainer.fit(classifier_model, train_loader, val_loader)


  | Name          | Type      | Params
--------------------------------------------
0 | loss_function | NLLLoss   | 0     
1 | conv1         | Conv2d    | 160   
2 | maxpool1      | MaxPool2d | 0     
3 | conv2         | Conv2d    | 4.6 K 
4 | maxpool2      | MaxPool2d | 0     
5 | flatten       | Flatten   | 0     
6 | fc1           | Linear    | 19.7 M
7 | dropout       | Dropout   | 0     
8 | fc2           | Linear    | 650   
--------------------------------------------
19.7 M    Trainable params
0         Non-trainable params
19.7 M    Total params
78.911    Total estimated model params size (MB)


/usr/local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 3:  19%|█▉        | 154/799 [01:47<07:25,  1.45it/s, loss=2.3, v_num=0, train_loss=2.300, train_acc=0.000, validation_loss=2.300, validation_acc=0.0921]

/usr/local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1051: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [493]:
best_model_path = checkpoint_callback.best_model_path

# Evaluation

In [494]:
test_dataset = GTZANDataset(csv_file='../processed_data/test_data.csv')

In [495]:
best_model = classifier_model.load_from_checkpoint(checkpoint_callback.best_model_path)

In [496]:
test_loader = DataLoader(test_dataset, num_workers=4)

In [497]:
trainer.test(model=best_model, test_dataloaders=test_loader)

/usr/local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:678: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  rank_zero_deprecation(


Testing:  99%|█████████▉| 197/199 [00:16<00:00, 18.94it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.37688443064689636, 'test_loss': 1.7891608476638794}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 199/199 [00:16<00:00, 12.10it/s]


[{'test_loss': 1.7891608476638794, 'test_acc': 0.37688443064689636}]